Burst Balloons
======

* [Problem Description](#problem_description)
* [Test Cases](#test_cases)
* [Approach 1: Trying all the possibilities](#approach_1)
* [Approach 2: Using memorization](#approach_2)
    * [A different perspective](#different_perspective)
    * [The bottom-up approach](#bottom_up_approach)
     * [4th level](#4th_level)
     * [3rd level](#3rd_level)
     * [2nd level](#2nd_level)
     * [1st level](#1st_level)
     * [Conclusion](#conclusion)

### Problem Description  <a id='problem_description'></a>

Given **n** balloons, indexed from **0** to **n-1**. Each balloon is painted with a number on it represented by array **nums**. You are asked to burst all the balloons. If the you burst balloon **i** you will get **nums[left] $*$ nums[i] $*$ nums[right]** coins. Here **left** and **right** are adjacent indices of **i**. After the burst, the **left** and **right** then becomes adjacent.

Find the maximum coins you can collect by bursting the balloons wisely.

Note:<br>
(1) You may imagine **nums[-1] = nums[n] = 1**. They are not real therefore you can not burst them.<br>
(2) 0 ≤ **n** ≤ 500, 0 ≤ **nums[i]** ≤ 100

Example:

Given: **[3, 1, 5, 8]**<br>
Return: **167**

Source: https://leetcode.com/problems/burst-balloons/description/

### Test Cases <a id='test_cases'></a>

I generated a few test cases so you can test your own implementations:

![](figures/burst_balloon_test_cases.png)

### Approach 1: Trying all the possibilities <a id='approach_1'></a>

One way of solving this problem is to try all possible ways of bursting the balloons and see which order maximizes the final score. This can also be viewed as a **top-bottom** approach.<br>
For the example presented by the problem (balloons [3, 1, 5, 8]), I created the following tree showing all possible cases:

![](figures/burst_balloon_ex1.png)

First we start with the balloons 3, 1, 5 and 8. <br>
We have 4 possible options of bursting the balloons. We could burst balloon #3 or balloon #1 or balloon #5 or balloon #8 as described in the first level from top down in the image.<br>
If we burst balloon 3 we will score 3 points, once $1*3*1=3$ (nums[left] $*$ nums[i] $*$ nums[right]). And as it is the first balloon we are bursting, the accumulated score is 3, shown as **acc** in the chart.<br>

**Observation:** According to Note (2), the first $1$ is used because as the balloon we are bursting is the most left in this sequence, there is no other balloon on its left, therefore we will always consider its left balloon as $1$. This same rule applies for bursting the most right balloon.

Then we are left with balloons 1, 5 and 8 as can be seen in the sequence. So we can continue exploring all the possible options of eliminating the balloons and in the end we observe all achievable final scores which are highlighted in the final branches of the tree.<br>
See that the highest possible score is **167** and can be obtained shooting the balloons 1, 5, 3 and 8 in sequence.

In this approach we will end up with $n!$ scores to decide which one is the highest with complexity equals to $O(n!)$.<br>
Where $n$ is the number of balloons we begin with. In this example we have $n=4$ resulting in $4!=24$ possible scores.<br>
Using this approach our algorithm has $O(24)$ operations.<p>

The code below shows my implementation of this approach.

In [ ]:
input = [ 3, 1, 5, 8] 
# You could also enter the balloons as: 
# input = '3158'

def operate(input, score, max_score):
    # Loop through each  balloon
    for i in range(len(input)):
        newinput = input[:i] + input[i+1:]
        # If you shoot most left balloon, the left balloon's score is 1 by default
        if i == 0: i_= 1 
        else: i_ = int(input[i-1])
        # If you shoot most right balloon, the right balloon's score is 1 by default
        if i == len(input)-1:  ii = 1
        else: ii = int(input[i+1])
        # Calculate score
        new_score = i_*int(input[i])*ii
        # Accumulated score
        acc = new_score + score
        # Uncomment this line to print all possible final scores
        # if (len(newinput) == 0): print('Final score:', acc)
        # If there is no more balloons to shoot, check if achieved score is the highest
        if ((len(newinput) == 0) & (acc > max_score)):
            max_score = acc
        else:
            # Go for next round
            max_score = operate(newinput, acc, max_score)
    return max_score

# Start bursting the balloons
score = operate(input, 0, 0)
# Showing the results
print('Max score:', score)    

## Approach 2: Using memorization<a id='approach_2'></a>

### A different perspective <a id='different_perspective'></a>

Another way to solve is to avoid calculating the same scores more than once.<br>
After popping each balloon a new branch of possibilities is produced. The image below highlights the new possible bursting options after each bursting

<a id='seq_bursting'></a>
![](figures/burst_balloon_ex2.png)

For the first bursting we have four possibilities (bursting balloon 3, 1, 5 or 8), leaving us with 4 possible balloons left:  [1,5,8], [3,5,8], [3,1,8], [3,1,5] (shown in red).
For the second bursting we have 12 possibilities: [5,8], [1,8], [1,5], [5,8], [3,8] ... [3,1] (shown in green).
For the 3rd bursting we have 24 possibilities: [8], [5], [8], [1], [5] ... [3] (shown in blue).
For the 4th and last bursting we have only one balloon left, leading us to 24 possibilities: [8], [5], [8], [1], [5] ... [3] (shown in yellow).

If we pay attention we can see that for the second bursting on, some of the options are repeated. The tree below shows that among all possible two-balloon options, we are left with balloons [5,8] twice.

![](figures/burst_balloon_ex3.png)

It is easily seen that other options of 2 balloons also repeats twice: [1,8], [1,5], [3,8] and [3,5]. It means that the operations we are performing in each of those branches are redundant! <br>
If we store those accumulated scores into memory we avoid unnecessary operations and improve our algorithm performance. It means we just have to calculate those scores once.

Each branch has its own accumulated scores. For example, if we consider the two branches with balloons [5,8], we see that it contributes with two gains each. If we burst balloon 5, the gain is 48 and if we burst balloon 8, the gain is 45 as shown below: 

![](figures/burst_balloon_ex4.png)

The gains are the same, because they are generated by popping up the same balloons.<br>

**As our goal is to obtain the highest possible score, we will only store the highest gain**, improving even more the efficiency of our algorithm.<br>
If we follow this appproach and complete the full tree, we will have:

<a id='full_tree_redundancy'></a>
![](figures/burst_balloon_ex5.png)

Now we just need to find the maximum value between all the maximum values: $max(56,53,59,52,53,78,167,75,73,59,96,91,54,53,49,75,49,51) = 167$

Now we got the idea of storing accumulated gains, we still can improve our algorithm by storing other repeated operations. From the [figure](#seq_bursting) and [figure](#full_tree_redundancy) we see that after the 3rd bursting we also have redundancies. <br>
See that during the 3rd burst the balloon [8] repeats 3 times, as well as with the other balloons [5],[1] and [3]. It means we are still having making more operations than we need.<br>
In order to maximize the efficiency of our algorithm, we will adopt the approach of storing the calculated values using a **bottom-up** approach.<br>

### The bottom-up approach<a id='bottom_up_approach'></a>

In this approach, is like if we are starting from the **4th burst** options and go up in the tree until we reach the **1st burst** options, as we saw on [figure](#seq_bursting).

We will consider bursting one balloon at a time as if we are starting from the bottom and we'll be moving up the tree:<br>
* First we will consider having 1 single balloon to burst (4th burst level).<br> 
* Then we will move one level in the three and consider having 2 balloons to burst (3rd burst level). <br> 
* Later we will go up one more level and consider having 3 balloons to burst (2nd burst level).<br> 
* Finally we will go to the highest level of the tree having 4 balloons to burst (1st burst level).

The advantage of our method is to store the values so we won't need to recalculate them. The table below will be used to store the values:

![](figures/burst_balloon_empty_table.png)


#### 4th level:<a id='4th_level'></a>

In this level we have only one balloon to burst. As the possible balloons to burst are [3], [1], [5] and [8], we have the following 4 options:

![](figures/burst_balloon_ex6.png)

We can complete the table with these calculated values:

<a id='table_4th_level'></a>
![](figures/burst_balloon_table_4th_level.png)

#### 3rd level:<a id='3rd_level'></a>

Now let's go up the tree to the 3rd level and calculate all the options as if we just have two balloons to burst.<br>

All the possible combinations for having two balloons to burst are:
* (C1) balloon [3] and balloon [1]: 
 * burst first balloon [3] and then balloon [1]
 * burst first balloon [1] and then balloon [3]
* (C2) balloon [1] and balloon [5]: 
 * burst first balloon [1] and then balloon [5]
 * burst first balloon [5] and then balloon [1]
* (C3) balloon [5] and balloon [8]: 
 * burst first balloon [5] and then balloon [8]
 * burst first balloon [8] and then balloon [5]
* (C4) balloon [3] and balloon [8]: 
 * burst first balloon [3] and then balloon [1]
 * burst first balloon [1] and then balloon [3]
* (C5) balloon [3] and balloon [5]: 
 * burst first balloon [3] and then balloon [5]
 * burst first balloon [5] and then balloon [3]
* (C6) balloon [1] and balloon [8]: 
 * burst first balloon [1] and then balloon [8]
 * burst first balloon [8] and then balloon [1]

There are important considerations to be made:
* There are $C(n,2)=\frac{n!}{2!(n-2)!}$ possible combinations if the order of bursting didn't matter. In our example with $n=4$, we will have $C(4,2)=6$ combinations in total. But as **the order of bursting matters**, we have to use Arrangement/Permutation. Therefore the total off possible combinations is given by $P(n,2)=\frac{n!}{(n-2)!}$. It will result in $P(4,2)=12$ combinations in total.
* Combinations C4 (balloons [3] and [8]) can only be achieved if the balloons [1] and [5] are burst first. In other words, C4 can only happen after C2.
* Combinations C5 (balloons [3] and [5]) can only be achieved if the balloon [1] is burst first. In other words, C5 can only happen after bursting balloon [1].
* Combinations C6 (balloons [1] and [8]) can only be achieved if the balloon [5] is burst first. In other words, C6 can only happen after bursting balloon [5].

As in the 3rd level we we are popping up two balloons, we will only need to evaluate the combinations C1, C2 and C3. Note that bursting a single balloon (balloon [1] and balloon [5] considered in C5 and C6 respectively) was already accounted in the 4th level.<br>

Thus in this 3rd level we will have:

![](figures/burst_balloon_ex7.png)

Notice that among the 12 operations we had to perform, we only needed to calculate 6 values. 6 operations we obtained consulting the [table](#table_4th_level) we used to store previous operations.

Let's complete our table storing only the max values. Why max values only? Because they are the only ones that will influence our final score:

<a id='table_4th_level'></a>
![](figures/burst_balloon_table_3rd_level.png)

#### 2nd level:<a id='2nd_level'></a>

Going up one level we reach the 2nd level of our tree.<br>
In this level we will consider having 3 balloons to burst.<br>
Using the permutation equation $P(n,3)=\frac{n!}{(n-3)!}$ , we can see that we have $P(4,3)=12$ possible combinations:
* (C1) balloons [3], [1] and [5]:
 * burst first balloon [3], then balloon [1] and then balloon [5]
 * burst first balloon [1], then balloon [3] and then balloon [5]
 * burst first balloon [3], then balloon [5] and then balloon [1]
 * burst first balloon [5], then balloon [3] and then balloon [1]
 * burst first balloon [5], then balloon [1] and then balloon [3]
 * burst first balloon [1], then balloon [5] and then balloon [3]
* (C2) balloons [1], [5] and [8]:
 * burst first balloon [1], then balloon [5] and then balloon [8]
 * burst first balloon [5], then balloon [1] and then balloon [8]
 * burst first balloon [1], then balloon [8] and then balloon [5]
 * burst first balloon [8], then balloon [1] and then balloon [5]
 * burst first balloon [5], then balloon [8] and then balloon [1]
 * burst first balloon [8], then balloon [5] and then balloon [1]

The image below illustrates all of the 4th level operations:

![](figures/burst_balloon_ex8.png)

As we make usage of our table of operations, in the 4th level we reduced our algorithm to only 8 operations.

Updating our table of operations, we have:

![](figures/burst_balloon_table_2nd_level.png)

#### 1st level:<a id='1st_level'></a>

Finally we have reached our 1st level of the tree. In this level we will consider having all the 4 balloons to burst.<br>
Using the permutation equation $P(n,4)=\frac{n!}{(n-4)!}$ , we can see that we have $P(4,4)=12$ possible combinations:
* (C1) balloons  [3], [1], [5] and [8]:
 * burst first balloon [3], then balloon [1], then balloon [5] and then balloon [8]
 * burst first balloon [3], then balloon [5], then balloon [1] and then balloon [8]
 * burst first balloon [1], then balloon [3], then balloon [5] and then balloon [8]
 * burst first balloon [1], then balloon [5], then balloon [3] and then balloon [8]
 * burst first balloon [5], then balloon [3], then balloon [1] and then balloon [8]
 * burst first balloon [5], then balloon [1], then balloon [3] and then balloon [8]
 * burst first balloon [1], then balloon [3], then balloon [8] and then balloon [5]
 * burst first balloon [3], then balloon [1], then balloon [8] and then balloon [5]
 * burst first balloon [3], then balloon [8], then balloon [1] and then balloon [5]
 * burst first balloon [8], then balloon [3], then balloon [1] and then balloon [5]
 * burst first balloon [1], then balloon [8], then balloon [3] and then balloon [5]
 * burst first balloon [8], then balloon [1], then balloon [3] and then balloon [5]
 * burst first balloon [3], then balloon [5], then balloon [8] and then balloon [1]
 * burst first balloon [3], then balloon [8], then balloon [5] and then balloon [1]
 * burst first balloon [5], then balloon [3], then balloon [8] and then balloon [1]
 * burst first balloon [5], then balloon [8], then balloon [3] and then balloon [1]
 * burst first balloon [8], then balloon [3], then balloon [5] and then balloon [1]
 * burst first balloon [8], then balloon [5], then balloon [3] and then balloon [1]
 * burst first balloon [1], then balloon [5], then balloon [8] and then balloon [3]
 * burst first balloon [1], then balloon [8], then balloon [5] and then balloon [3]
 * burst first balloon [5], then balloon [1], then balloon [8] and then balloon [3]
 * burst first balloon [5], then balloon [8], then balloon [1] and then balloon [3]
 * burst first balloon [8], then balloon [1], then balloon [5] and then balloon [3]
 * burst first balloon [8], then balloon [5], then balloon [1] and then balloon [3]
 
Calculating all these operations, we have:

![](figures/burst_balloon_ex9.png)

As we make usage of our table of operations, in this level we reduced our algorithm to only 4 operations.

Updating our table of operations, we have:

![](figures/burst_balloon_table_1st_level.png)

#### Conclusion:<a id='conclusion'></a>

We saw step-by-step that applying **memorization** helps us to obtain the same result eliminating repeated operations. This approach is obtained using a **bottom-up** perspective.<p>
In the previous code our complexity was in a order of $O(n!)$. With this new context we reduced our time complexity to **$O(n^3)$**

The code below shows the implementation of this approach:

In [ ]:
class Solution(object):
    def maxCoins(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        nums = [1] + nums + [1]
        n = len(nums)
        dp = [[0] * n for _ in range(n)]
        def calculate(i, j):
            if dp[i][j] or j == i + 1: # in memory or gap < 2
                return dp[i][j]
            coins = 0
            for k in range(i+1, j): # find the last balloon
                coins = max(coins, nums[i] * nums[k] * nums[j] + calculate(i, k) + calculate(k, j))
            dp[i][j] = coins
            return coins

        return calculate(0, n-1)

sol = Solution()

# import random

# for j in range(4):
#     n = 9
#     array = []
#     for i in range(n):
#         array.append(random.randint(1, 10))

#     res  = sol.maxCoins(nums=array)
#     print('Input: %s Output: %s' % (array, res))


sol.maxCoins(nums=[3,1,5,8] )

In [7]:
def operate(input):
    input = [1] + input + [1]
    n = len(input)
    # Create list of n+2 arrays of zeros, each with length = n+2
    dp = [[0] * n for _ in range(n)]
    
    def calculate_scores(i, j):
        print('entrou')

        if dp[i][j] or j == i + 1: # in memory or gap < 2
            return dp[i][j]
        coins = 0
        for k in range(i+1, j): # find the last balloon
            coins = max(coins, input[i] * input[k] * input[j] + calculate_scores(i, k) + calculate_scores(k, j))
        dp[i][j] = coins
        return coins

    # Calculate scores
    print(n-1)
    return calculate_scores(0, n-1)



operate([3,2,1] )

4
entrou
entrou
entrou
entrou
entrou
entrou
entrou
entrou
entrou
entrou
entrou
entrou
entrou
entrou
entrou
entrou
entrou
entrou
entrou
entrou
entrou


12

In [ ]:
input = [ 3, 1, 5, 8] 
# You could also enter the balloons as: 
# input = '3158'

def operate(input, score, max_score):
    # Loop through each  balloon
    for i in range(len(input)):
        newinput = input[:i] + input[i+1:]
        # If you shoot most left balloon, the left balloon's score is 1 by default
        if i == 0: i_= 1 
        else: i_ = int(input[i-1])
        # If you shoot most right balloon, the right balloon's score is 1 by default
        if i == len(input)-1:  ii = 1
        else: ii = int(input[i+1])
        # Calculate score
        new_score = i_*int(input[i])*ii
        # Accumulated score
        acc = new_score + score
        # Uncomment this line to print all possible final scores
        # if (len(newinput) == 0): print('Final score:', acc)
        # If there is no more balloons to shoot, check if achieved score is the highest
        if ((len(newinput) == 0) & (acc > max_score)):
            max_score = acc
        else:
            # Go for next round
            max_score = operate(newinput, acc, max_score)
    return max_score

# Start bursting the balloons
score = operate(input, 0, 0)
# Showing the results
print('Max score:', score)    

In [ ]:
input = [ 3, 1, 5, 8] 
# You could also enter the balloons as: 
# input = '3158'

def operate(input, memory, length, acc_score):
    # Loop through each  balloon
    for i in range(len(input)):
        newinput = input[:i] + input[i+1:]
        print('%s => %s' % (input, newinput))
        
        ##### CALCULATE SCORE
        # If you shoot most left balloon, the left balloon's score is 1 by default
        if i == 0: i_= 1 
        else: i_ = int(input[i-1])
        # If you shoot most right balloon, the right balloon's score is 1 by default
        if i == len(input)-1:  ii = 1
        else: ii = int(input[i+1])
        # Calculate score
        new_score = i_*int(input[i])*ii
        acc_score = acc_score+new_score
        #####

        #memory.append([input,i,new_score, acc_score])
        print('----MEMORY----')
        for j in range(len(memory)):
            print(memory[j])
        print('--------------')
        for j in range(len(newinput)):
            #teste = newinput[:j] + newinput[j+1:]
            print(newinput)
            acumulado = operate(newinput, memory, length+1, acc_score)
            print(acumulado)
        
    return [input,i,new_score]

memory = []
# Start bursting the balloons
operate(input, memory, 1, 0)
# Showing the results
#print('Max score:', score)    
